In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
import os

In [2]:
from langchain_groq import ChatGroq
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018237BFC220>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018237BFD690>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100,
)

final_docs = text_splitter.split_documents(docs)
final_docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [5]:
len(final_docs)

13

In [6]:
### Prompt Template

chunk_prompt = """

Please summarize the below speech:
Speech: `{text}`
Summary: 

"""

map_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=chunk_prompt,
)

In [8]:
final_prompt = """ 

Provide the final summary of the entire speech with these important points.
Add a Motivation Title, Start the precise summary with an introduction and provide the summary in number points for the speech.

Speech: {text}

"""

final_prompt_template = PromptTemplate(
    input_variables=["text"],
    template=final_prompt,
)

In [9]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True,
)

output = summary_chain.run(final_docs)
print(output)

C:\Users\adity\AppData\Local\Temp\ipykernel_17380\3428168995.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = summary_chain.run(final_docs)




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Please summarize the below speech:
Speech: `A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have 

c:\Users\adity\Desktop\GEN AI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (2428 > 1024). Running this sequence through the model will result in indexing errors




> Entering new LLMChain chain...
Prompt after formatting:
 

Provide the final summary of the entire speech with these important points.
Add a Motivation Title, Start the precise summary with an introduction and provide the summary in number points for the speech.

Speech: In his departing speech, A.P.J. Abdul Kalam expresses gratitude for the opportunity to serve as President of India for five years. He highlights the diverse groups he interacted with during his tenure, emphasizing the importance of each sector in national development.  

Kalam then outlines ten key messages from his experiences:

1. **Accelerate Development:** Fueled by the aspirations of India's youth.
2. **Empower Villages:** Recognizing their crucial role.
3. **Mobilize Rural Competence:** Leveraging local strengths for competitiveness.
4. **Seed to Food:** Strengthening agricultural growth.
5. **Defeat Problems and Succeed:** Embracing a spirit of resilience.
6. **Partnership Over Problems:** Collaborative solu